# Blackjack AR

In [1]:
#cartas de poker
class Card:
    def __init__(self, suit, value, image):
        self.suit = suit
        self.value = value
        self.image = image
        self.inGame = False

In [2]:
def init_cartas():
    for suit in suits:
        for value in values:
            card = Card(suit, value, f'cartas/{value}_of_{suit}.png')
            objects.append(card)

suits = ['clubs', 'diamonds', 'heards', 'spades']
values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king', 'ace']
objects = []

init_cartas()

In [3]:
from __future__ import print_function # Python 2/3 compatibility
import cv2 # Import the OpenCV library
import numpy as np # Import Numpy library
import mediapipe as mp
import sys

# Configuración de MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

desired_aruco_dictionary = "DICT_4X4_100"
 
# The different ArUco dictionaries built into the OpenCV library. 
ARUCO_DICT = {
  "DICT_4X4_50": cv2.aruco.DICT_4X4_50,
  "DICT_4X4_100": cv2.aruco.DICT_4X4_100,
  "DICT_4X4_250": cv2.aruco.DICT_4X4_250,
  "DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
  "DICT_5X5_50": cv2.aruco.DICT_5X5_50,
  "DICT_5X5_100": cv2.aruco.DICT_5X5_100,
  "DICT_5X5_250": cv2.aruco.DICT_5X5_250,
  "DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
  "DICT_6X6_50": cv2.aruco.DICT_6X6_50,
  "DICT_6X6_100": cv2.aruco.DICT_6X6_100,
  "DICT_6X6_250": cv2.aruco.DICT_6X6_250,
  "DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
  "DICT_7X7_50": cv2.aruco.DICT_7X7_50,
  "DICT_7X7_100": cv2.aruco.DICT_7X7_100,
  "DICT_7X7_250": cv2.aruco.DICT_7X7_250,
  "DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
  "DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL
}

In [4]:
def showArucoDetections(corners, ids, frame):
    # Check that at least one ArUco marker was detected
    if len(corners) > 0:
      # Flatten the ArUco IDs list
      ids = ids.flatten()
       
      # Loop over the detected ArUco corners
      for (marker_corner, marker_id) in zip(corners, ids):
       
        # Extract the marker corners
        corners = marker_corner.reshape((4, 2))
        (top_left, top_right, bottom_right, bottom_left) = corners
         
        # Convert the (x,y) coordinate pairs to integers
        top_right = (int(top_right[0]), int(top_right[1]))
        bottom_right = (int(bottom_right[0]), int(bottom_right[1]))
        bottom_left = (int(bottom_left[0]), int(bottom_left[1]))
        top_left = (int(top_left[0]), int(top_left[1]))
         
        # Draw the bounding box of the ArUco detection
        cv2.line(frame, top_left, top_right, (0, 255, 0), 2)
        cv2.line(frame, top_right, bottom_right, (0, 255, 0), 2)
        cv2.line(frame, bottom_right, bottom_left, (0, 255, 0), 2)
        cv2.line(frame, bottom_left, top_left, (0, 255, 0), 2)
         
        # Calculate and draw the center of the ArUco marker
        center_x = int((top_left[0] + bottom_right[0]) / 2.0)
        center_y = int((top_left[1] + bottom_right[1]) / 2.0)
        cv2.circle(frame, (center_x, center_y), 4, (0, 0, 255), -1)
         
        # Draw the ArUco marker ID on the video frame
        # The ID is always located at the top_left of the ArUco marker
        cv2.putText(frame, "ARUCO: " + str(marker_id) + "Starting game", 
          (top_left[0], top_left[1] - 15),
          cv2.FONT_HERSHEY_SIMPLEX,
          0.5, (0, 255, 0), 2)


In [5]:
def count_fingers_up(hand_landmarks):
    """
    Cuenta el número total de dedos levantados en la mano desde la vista dorsal.
    
    Args:
        hand_landmarks: Landmarks de la mano detectada por MediaPipe
    
    Returns:
        int: Número total de dedos levantados
        list: Estado de cada dedo (True si está levantado, False si no)
    """
    # Puntos de referencia para las puntas de los dedos
    finger_tips = [4, 8, 12, 16, 20]  # Pulgar, índice, medio, anular, meñique
    
    # Puntos de referencia para las articulaciones base de los dedos
    finger_bases = [2, 5, 9, 13, 17]  # Base del pulgar y nudillos de los demás dedos
    
    fingers_up = []

    # Detectar si es la mano derecha o izquierda usando la posición relativa de la muñeca (landmark 0) y el CMC del pulgar (landmark 2)
    wrist_x = hand_landmarks.landmark[0].x
    thumb_cmc_x = hand_landmarks.landmark[2].x
    is_right_hand = thumb_cmc_x < wrist_x  # Mano derecha si el CMC del pulgar está a la izquierda del eje de la muñeca

    # Comparación para el pulgar desde la vista dorsal
    thumb_tip_x = hand_landmarks.landmark[4].x
    thumb_ip_x = hand_landmarks.landmark[3].x  # Articulación interfalángica
    thumb_base_x = hand_landmarks.landmark[2].x

    if is_right_hand:
        # Para la mano derecha: el pulgar está levantado si su punta está a la derecha de su base
        fingers_up.append(thumb_tip_x > thumb_ip_x)
    else:
        # Para la mano izquierda: el pulgar está levantado si su punta está a la izquierda de su base
        fingers_up.append(thumb_tip_x < thumb_ip_x)

    # Comparación para los demás dedos (vista dorsal)
    for tip, base in zip(finger_tips[1:], finger_bases[1:]):
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[base].y:  # La punta del dedo está más arriba que la base
            fingers_up.append(True)
        else:
            fingers_up.append(False)

    # Contar el total de dedos levantados
    total_fingers = sum(fingers_up)

    return total_fingers, fingers_up, is_right_hand

In [7]:
def main():
  """
  Main method of the program.
  """
  # Check that we have a valid ArUco marker
  if ARUCO_DICT.get(desired_aruco_dictionary, None) is None:
    print("[INFO] ArUCo tag of '{}' is not supported".format(
      args["type"]))
    sys.exit(0)
     
  # Load the ArUco dictionary
  print("[INFO] detecting '{}' markers...".format(
    desired_aruco_dictionary))
  

  this_aruco_dictionary = cv2.aruco.Dictionary_get(ARUCO_DICT[desired_aruco_dictionary])
  this_aruco_parameters = cv2.aruco.DetectorParameters_create()
   
  # Start the video stream
  cap = cv2.VideoCapture(0)
   
  with mp_hands.Hands(
          model_complexity=0,
          min_detection_confidence=0.5,
          min_tracking_confidence=0.5) as hands:
          
          # Iniciar la captura de video
          cap = cv2.VideoCapture(0)
          
          while True:
              # Capturar frame
              ret, frame = cap.read()
              if not ret:
                  break
                  
              # Convertir el frame a RGB para MediaPipe
              frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
              
              # Procesar el frame para detectar manos
              results = hands.process(frame_rgb)
              
              # Detect ArUco markers in the video frame
              (corners, ids, rejected) = cv2.aruco.detectMarkers(frame, this_aruco_dictionary, parameters=this_aruco_parameters)
              
              # Mostrar detecciones de ArUco
              showArucoDetections(corners, ids, frame)
              
              # Dibujar las detecciones de manos
              if results.multi_hand_landmarks:
                  for hand_landmarks in results.multi_hand_landmarks:
                      mp_drawing.draw_landmarks(
                          frame,
                          hand_landmarks,
                          mp_hands.HAND_CONNECTIONS,
                          mp_drawing_styles.get_default_hand_landmarks_style(),
                          mp_drawing_styles.get_default_hand_connections_style())
                      
                      # Contar dedos levantados
                      num_fingers, fingers_status, is_right_hand = count_fingers_up(hand_landmarks)
                      
                      # Mostrar el número de dedos en pantalla
                      cv2.putText(frame, f"Dedos: {num_fingers}", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                      
                      cv2.putText(frame, f"Mano derecha: {is_right_hand}", (10, 210),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                      
                      # Mostrar estado de cada dedo
                      finger_names = ["Pulgar", "Indice", "Medio", "Anular", "Menique"]
                      for i, (finger, is_up) in enumerate(zip(finger_names, fingers_status)):
                          status = "Arriba" if is_up else "Abajo"
                          cv2.putText(frame, f"{finger}: {status}", (10, 60 + (30 * i)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                          
              
              # Mostrar el frame
              cv2.imshow('ArUco + Hand Detection', frame)
              
              # Salir con 'q'
              if cv2.waitKey(1) & 0xFF == ord('q'):
                  break
          
          # Liberar recursos
          cap.release()
          cv2.destroyAllWindows()
   
if __name__ == '__main__':
  print(__doc__)
  main()

Automatically created module for IPython interactive environment
[INFO] detecting 'DICT_4X4_100' markers...
